In [12]:
# ✅ Install dependencies (run once)
# !pip install -q transformers torch sentencepiece openpyxl pandas tqdm

import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm
import time

# ✅ Use lighter model to prevent crashes
model_name = "google/flan-t5-base"  # MUCH lighter than flan-t5-xl
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ✅ Send to device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ✅ Load Excel file
input_path = r"C:\Users\HP\Downloads\501-1000disc.xlsx"
df = pd.read_excel(input_path)

# ✅ Add rephrased_description column if missing
if "rephrased_description" not in df.columns:
    df["rephrased_description"] = ""

# ✅ Store results here
rephrased_list = []

# ✅ Rephrase descriptions
print("🎬 Rephrasing with FLAN-T5-Base...")

for i in tqdm(range(len(df)), desc="Processing"):
    original = str(df.at[i, "description"]).strip()

    # Skip if already rephrased
    if pd.isna(df.at[i, "rephrased_description"]) or df.at[i, "rephrased_description"] == "":
        prompt = f"Paraphrase this movie description in a single simple sentence:\n\n{original}"
        try:
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
            outputs = model.generate(**inputs, max_new_tokens=100)
            new_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        except Exception as e:
            print(f"❌ Error at row {i}: {e}")
            new_desc = original  # fallback

        rephrased_list.append(new_desc)
    else:
        rephrased_list.append(df.at[i, "rephrased_description"])

    time.sleep(0.2)  # Light delay for stability

# ✅ Update DataFrame and save once at end
df["rephrased_description"] = rephrased_list
df.to_excel(input_path, index=False)

print("✅ All rephrased descriptions saved to Excel.")

ModuleNotFoundError: Could not import module 'BlenderbotConfig'. Are this object's requirements defined correctly?